In [71]:

import logging
logging.basicConfig()
logger = logging.getLogger('')
logger.setLevel(logging.INFO)

logger.debug("debug")

In [79]:
# %load sudoku.py
import pandas as pd
import numpy as np
def getPuzzle(puzale_str):
    return np.array(list(puzale_str)).reshape(9,9)

def isSolved(np_cells):
    list=np_cells.reshape(81)
    logger.debug(list)
    return not ('0' in list)

# def map2orginal(row, col, block_row, block_col):
#     return (block_row*3 +row, block_col*3 + col)

def getBlockIndex(row_index, col_index):
    r= row_index//3
    c = col_index//3
    return (r,c)

def getBlock(block_row, block_col, np_cells):
    return np_cells[block_row*3:block_row*3+3, block_col*3:block_col*3+3]

def getBlockList(block_row, block_col, np_cells):
    return getBlock(block_row, block_col, np_cells).reshape(9)

def getRow(row, np_cells):
    return np_cells[row]

def getCol(col, np_cells):
    return np_cells[:, col]

def calculate(row_index, col_index, np_cells):
    row_index=row_index
    col_index=col_index
    row=getRow(row_index, np_cells)
    col=getCol(col_index, np_cells)
    block_row, block_col = getBlockIndex(row_index, col_index)
    block=getBlockList(block_row, block_col, np_cells)
    possibilities={i for i in "123456789"}
    for i in row:
        if i in possibilities:
            possibilities.remove(i)
    for j in col:
        if j in possibilities:
            possibilities.remove(j)
    for k in block:
        if k in possibilities:
            possibilities.remove(k)
    return (row_index, col_index, possibilities)

def calculate_cells(np_cells):
    solved=True
    changed=True
    results=[]
    iterations=0
    logger.debug(np_cells)
    while changed:
        iterations+=1
        changed=False
        results=[]
        for i in range(9):
            for j in range(9):
                if np_cells[i,j] == '0' :
                    results.append(calculate(i,j,np_cells))
                    solved=False


        for x,y,s in results:
            if len(s)==1:
                np_cells[x,y]=next(iter(s))
                changed=True
        
        logger.debug("results: {}".format(results))
        logger.debug("iterations: {}".format(iterations) )
        logger.debug("change: {}".format(changed) )

    logger.debug(np_cells)
    return solved


class Block:
    def __init__(self, r, c, np_cells) -> None:
        self.block=getBlock(r,c,np_cells)
        logger.debug(self.block)
        self.block_row, self.block_col=(r,c)
        self.np_cells=np_cells
        self.missing_set={i for i in "123456789"}
        self.missings={}
        self.unknowns=[]
        for i in range(3):
            for j in range(3):
                if self.block[i,j]=='0':
                    self.unknowns.append((i,j))
                else:
                    self.missing_set.remove(self.block[i,j])
        logger.debug("unknown: {}".format(self.unknowns) )
        logger.debug(self.missing_set)
        logger.debug(self.missings)

        for m in self.missing_set:
            for u in self.unknowns:
                if self.couldfit(m, u):
                    logger.debug("add {} to {}".format(u, m))
                    self.addpos(m, u)
                else:
                    logger.debug("could not add {} to {}".format(u, m))
        
        logger.debug("unknown: {}".format(self.unknowns) )
        logger.debug(self.missing_set)
        logger.debug(self.missings)

    def getMissings(self, missing_set, unknowns):
        pass

    def couldfit(self, m, u):
        m_r=u[0]+self.block_row*3
        m_c=u[1]+self.block_col*3
        row=getRow(m_r, self.np_cells)
        #print("row:", row, " m_r:", m_r)
        col=getCol(m_c, self.np_cells)
        #print("col:",col, " m_c:", m_c)
        result =True
        if m in row:
            logger.debug("found {} in {}".format(m, row))
            result=False
        else:
            logger.debug("No {} in row {}: {}".format(m, m_r, row))
        if m in col:
            logger.debug("found {} in {}".format(m, col))
            result=False
        else:
            logger.debug("No {} in col {}: {}".format(m,m_c, col))
       
        return result

    def addpos(self, m, u):
        if m in self.missings.keys():
            poss=self.missings[m]
            poss.append(u)
        else:
            poss=[u]
            self.missings[m]=poss

    def removepos(self,removes):
        deleted=[]
        for r in removes:
            for m in self.missings.keys():
                if r in self.missings[m]:
                    self.missings[m].remove(r)
                    if len(self.missings[m])==0:
                        deleted.append(m)
                    
        for d in deleted:
            self.missings.pop(d)


    def calculate(self):
        removes=[]
        changed=False
        repeat=True
        while repeat:
            repeat=False
            for m in self.missings.keys():
                if len(self.missings[m]) == 1:
                    px, py=self.missings[m][0]
                    logger.debug("Solved {} at {}".format(m,self.missings[m][0]))
                    self.block[px, py]=m
                    removes.append((px,py))
                    logger.debug(self.block)
                    changed=True
                    repeat=False
            self.removepos(removes)
        
        return changed
    
def calculate_blocks(np_cells):
    changed=False
    for block_row in range(3):
        for block_col in range(3):
            logger.debug("solving {},{}".format(block_row, block_col) )
            b=Block(block_row, block_col, np_cells)
            if b.calculate():
                changed=True
    return changed

def single_solve(np_cells):
    changed=True
    solved=False
    while changed:
        changed=False
        if calculate_cells(np_cells):
            solved=True
        else:
            if calculate_blocks(np_cells):
                changed=True
        if not solved:
            solved=isSolved(np_cells)

    logger.debug(np_cells)
    return solved

logger.debug('info')


In [88]:
import unittest

puzzle_str='070000043040009610800634900094052000358460020000800530080070091902100005007040802'
puzzle=getPuzzle(puzzle_str)
solution_str='679518243543729618821634957794352186358461729216897534485276391962183475137945862'
solution=getPuzzle(solution_str)
class TestNotebook(unittest.TestCase):
    
    def test_getPuzzle(self):
        expected=np.array(
      [['0', '7', '0', '0', '0', '0', '0', '4', '3'],
       ['0', '4', '0', '0', '0', '9', '6', '1', '0'],
       ['8', '0', '0', '6', '3', '4', '9', '0', '0'],
       ['0', '9', '4', '0', '5', '2', '0', '0', '0'],
       ['3', '5', '8', '4', '6', '0', '0', '2', '0'],
       ['0', '0', '0', '8', '0', '0', '5', '3', '0'],
       ['0', '8', '0', '0', '7', '0', '0', '9', '1'],
       ['9', '0', '2', '1', '0', '0', '0', '0', '5'],
       ['0', '0', '7', '0', '4', '0', '8', '0', '2']])
        
        self.assertTrue(np.array_equal(puzzle,expected))
    
    def test_solved(self):
         self.assertEqual(isSolved(puzzle),False)
    
    def test_unsolved(self):
         self.assertEqual(isSolved(solution),True)

    def test_getBlockIndex(self):
         self.assertEqual(getBlockIndex(8,8),(2,2))

    def test_getBlock(self):
         expected=np.array(
              [['0','0','0'],
               ['0','2','0'],
               ['5','3','0']])
         result=getBlock(1,2,puzzle)
         self.assertTrue(np.array_equal(result,expected))
    
    def test_getBlockList(self):
         expected=np.array(['0','0','0','0','2','0','5','3','0'])
         result=getBlockList(1,2,puzzle)
         self.assertTrue(np.array_equal(result,expected))

unittest.main(argv=[''], verbosity=2, exit=False)

test_getBlock (__main__.TestNotebook.test_getBlock) ... ok
test_getBlockIndex (__main__.TestNotebook.test_getBlockIndex) ... ok
test_getBlockList (__main__.TestNotebook.test_getBlockList) ... ERROR
test_getPuzzle (__main__.TestNotebook.test_getPuzzle) ... ok
test_solved (__main__.TestNotebook.test_solved) ... ok
test_unsolved (__main__.TestNotebook.test_unsolved) ... ok

ERROR: test_getBlockList (__main__.TestNotebook.test_getBlockList)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/rk/34w_p7d95pl2jtrgpgfvyjvc0000gq/T/ipykernel_95611/2899059895.py", line 41, in test_getBlockList
    expected=np.array(
             ^^^^^^^^^
TypeError: array() takes from 1 to 2 positional arguments but 9 were given

----------------------------------------------------------------------
Ran 6 tests in 0.005s

FAILED (errors=1)


In [50]:
df = pd.read_csv("./data/sudoku.csv")

In [80]:
puzzle_str=df["puzzle"].loc(0)[0]
print(puzzle_str)
solution_str=df["solution"].loc(0)[0]
print(solution_str)
np_solution=np.array(list(solution_str)).reshape(9,9)
print()
print(np_solution)



070000043040009610800634900094052000358460020000800530080070091902100005007040802
679518243543729618821634957794352186358461729216897534485276391962183475137945862

[['6' '7' '9' '5' '1' '8' '2' '4' '3']
 ['5' '4' '3' '7' '2' '9' '6' '1' '8']
 ['8' '2' '1' '6' '3' '4' '9' '5' '7']
 ['7' '9' '4' '3' '5' '2' '1' '8' '6']
 ['3' '5' '8' '4' '6' '1' '7' '2' '9']
 ['2' '1' '6' '8' '9' '7' '5' '3' '4']
 ['4' '8' '5' '2' '7' '6' '3' '9' '1']
 ['9' '6' '2' '1' '8' '3' '4' '7' '5']
 ['1' '3' '7' '9' '4' '5' '8' '6' '2']]


In [74]:
np_cells=np.array(list(puzzle_str)).reshape(9,9)
np_cells

array([['0', '7', '0', '0', '0', '0', '0', '4', '3'],
       ['0', '4', '0', '0', '0', '9', '6', '1', '0'],
       ['8', '0', '0', '6', '3', '4', '9', '0', '0'],
       ['0', '9', '4', '0', '5', '2', '0', '0', '0'],
       ['3', '5', '8', '4', '6', '0', '0', '2', '0'],
       ['0', '0', '0', '8', '0', '0', '5', '3', '0'],
       ['0', '8', '0', '0', '7', '0', '0', '9', '1'],
       ['9', '0', '2', '1', '0', '0', '0', '0', '5'],
       ['0', '0', '7', '0', '4', '0', '8', '0', '2']], dtype='<U1')

In [54]:
np_cells=np.array(list(puzzle_str)).reshape(9,9)
np_cells
#single_solve(np_cells)
b=getBlock(0,0, np_cells)
print(b)
# getBlockArray(0,0, np_cells)
getBlockIndex(3,3)

[['0' '7' '0']
 ['0' '4' '0']
 ['8' '0' '0']]


(1, 1)

In [56]:


np_cells=np.array(list(puzzle_str)).reshape(9,9)
print(np_cells)
b=Block(1,1,np_cells)

print(np_cells)
print(b.missing_set)
print(b.missings)
b.calculate()
print(b.missings)
print(np_cells)

[['0' '7' '0' '0' '0' '0' '0' '4' '3']
 ['0' '4' '0' '0' '0' '9' '6' '1' '0']
 ['8' '0' '0' '6' '3' '4' '9' '0' '0']
 ['0' '9' '4' '0' '5' '2' '0' '0' '0']
 ['3' '5' '8' '4' '6' '0' '0' '2' '0']
 ['0' '0' '0' '8' '0' '0' '5' '3' '0']
 ['0' '8' '0' '0' '7' '0' '0' '9' '1']
 ['9' '0' '2' '1' '0' '0' '0' '0' '5']
 ['0' '0' '7' '0' '4' '0' '8' '0' '2']]
[['0' '7' '0' '0' '0' '0' '0' '4' '3']
 ['0' '4' '0' '0' '0' '9' '6' '1' '0']
 ['8' '0' '0' '6' '3' '4' '9' '0' '0']
 ['0' '9' '4' '0' '5' '2' '0' '0' '0']
 ['3' '5' '8' '4' '6' '0' '0' '2' '0']
 ['0' '0' '0' '8' '0' '0' '5' '3' '0']
 ['0' '8' '0' '0' '7' '0' '0' '9' '1']
 ['9' '0' '2' '1' '0' '0' '0' '0' '5']
 ['0' '0' '7' '0' '4' '0' '8' '0' '2']]
{'7', '1', '9', '3'}
{'7': [(0, 0), (1, 2), (2, 2)], '1': [(1, 2), (2, 1), (2, 2)], '9': [(2, 1)], '3': [(0, 0)]}
{'7': [(1, 2), (2, 2)], '1': [(1, 2), (2, 2)]}
[['0' '7' '0' '0' '0' '0' '0' '4' '3']
 ['0' '4' '0' '0' '0' '9' '6' '1' '0']
 ['8' '0' '0' '6' '3' '4' '9' '0' '0']
 ['0' '9' '4' '3' 

In [62]:
np_cells=np.array(list(puzzle_str)).reshape(9,9)
single_solve(np_cells)

INFO:root:[['6' '7' '9' '5' '1' '8' '2' '4' '3']
 ['5' '4' '3' '7' '2' '9' '6' '1' '8']
 ['8' '2' '1' '6' '3' '4' '9' '5' '7']
 ['7' '9' '4' '3' '5' '2' '1' '8' '6']
 ['3' '5' '8' '4' '6' '1' '7' '2' '9']
 ['2' '1' '6' '8' '9' '7' '5' '3' '4']
 ['4' '8' '5' '2' '7' '6' '3' '9' '1']
 ['9' '6' '2' '1' '8' '3' '4' '7' '5']
 ['1' '3' '7' '9' '4' '5' '8' '6' '2']]


True

In [59]:
np_cells=np.array(list(puzzle_str)).reshape(9,9)
calculate_cells(np_cells)
b00=Block(0,1,np_cells)
b00.calculate()
print(b.missings)
print(np_cells)

# b01=Block(0,1,np_cells)
# b01.calculate()
# print(b.missings)
# print(np_cells)

INFO:root:[['0' '7' '0' '0' '0' '0' '0' '4' '3']
 ['0' '4' '0' '0' '0' '9' '6' '1' '0']
 ['8' '0' '0' '6' '3' '4' '9' '0' '0']
 ['0' '9' '4' '0' '5' '2' '0' '0' '0']
 ['3' '5' '8' '4' '6' '0' '0' '2' '0']
 ['0' '0' '0' '8' '0' '0' '5' '3' '0']
 ['0' '8' '0' '0' '7' '0' '0' '9' '1']
 ['9' '0' '2' '1' '0' '0' '0' '0' '5']
 ['0' '0' '7' '0' '4' '0' '8' '0' '2']]


{'7': [(1, 2), (2, 2)], '1': [(1, 2), (2, 2)]}
[['0' '7' '0' '0' '0' '0' '0' '4' '3']
 ['0' '4' '0' '7' '0' '9' '6' '1' '0']
 ['8' '0' '0' '6' '3' '4' '9' '0' '0']
 ['0' '9' '4' '0' '5' '2' '0' '0' '0']
 ['3' '5' '8' '4' '6' '0' '0' '2' '0']
 ['0' '0' '0' '8' '0' '0' '5' '3' '0']
 ['0' '8' '0' '0' '7' '0' '0' '9' '1']
 ['9' '0' '2' '1' '0' '0' '0' '0' '5']
 ['0' '0' '7' '0' '4' '0' '8' '0' '2']]


In [60]:
print(np_cells)
c=getBlock(0,1,np_cells)
c[1,0]='x'
print(np_cells)

[['0' '7' '0' '0' '0' '0' '0' '4' '3']
 ['0' '4' '0' '7' '0' '9' '6' '1' '0']
 ['8' '0' '0' '6' '3' '4' '9' '0' '0']
 ['0' '9' '4' '0' '5' '2' '0' '0' '0']
 ['3' '5' '8' '4' '6' '0' '0' '2' '0']
 ['0' '0' '0' '8' '0' '0' '5' '3' '0']
 ['0' '8' '0' '0' '7' '0' '0' '9' '1']
 ['9' '0' '2' '1' '0' '0' '0' '0' '5']
 ['0' '0' '7' '0' '4' '0' '8' '0' '2']]
[['0' '7' '0' '0' '0' '0' '0' '4' '3']
 ['0' '4' '0' 'x' '0' '9' '6' '1' '0']
 ['8' '0' '0' '6' '3' '4' '9' '0' '0']
 ['0' '9' '4' '0' '5' '2' '0' '0' '0']
 ['3' '5' '8' '4' '6' '0' '0' '2' '0']
 ['0' '0' '0' '8' '0' '0' '5' '3' '0']
 ['0' '8' '0' '0' '7' '0' '0' '9' '1']
 ['9' '0' '2' '1' '0' '0' '0' '0' '5']
 ['0' '0' '7' '0' '4' '0' '8' '0' '2']]


In [68]:
tests=df['puzzle'].loc[:100]
for t in tests:
    np_cells=np.array(list(t)).reshape(9,9)
    if not single_solve(np_cells):
        print("Not solve: {}".format(np_cells))

Not solve: [['9' '0' '4' '5' '2' '1' '6' '7' '0']
 ['0' '6' '1' '8' '9' '7' '2' '4' '0']
 ['7' '0' '2' '6' '4' '3' '0' '0' '1']
 ['1' '7' '0' '9' '6' '0' '3' '8' '0']
 ['0' '0' '0' '1' '3' '8' '7' '0' '6']
 ['6' '0' '0' '2' '7' '0' '0' '1' '0']
 ['0' '9' '0' '4' '8' '6' '1' '0' '7']
 ['0' '1' '6' '7' '5' '9' '0' '3' '0']
 ['0' '0' '7' '3' '1' '2' '5' '6' '9']]
Not solve: [['9' '2' '5' '3' '4' '6' '1' '7' '8']
 ['7' '8' '6' '2' '9' '1' '3' '5' '4']
 ['0' '0' '0' '8' '7' '5' '2' '6' '9']
 ['5' '6' '2' '9' '3' '8' '7' '4' '1']
 ['0' '7' '0' '5' '6' '2' '0' '0' '3']
 ['0' '0' '0' '7' '1' '4' '5' '2' '6']
 ['0' '9' '0' '1' '5' '7' '6' '3' '2']
 ['2' '0' '7' '6' '8' '0' '4' '0' '5']
 ['6' '5' '0' '4' '2' '0' '0' '0' '7']]
Not solve: [['6' '2' '0' '7' '4' '5' '1' '0' '0']
 ['0' '7' '4' '1' '0' '0' '6' '5' '2']
 ['5' '1' '8' '0' '0' '0' '3' '7' '4']
 ['0' '6' '7' '3' '0' '0' '9' '0' '0']
 ['0' '9' '0' '0' '0' '0' '0' '6' '0']
 ['8' '4' '0' '9' '7' '6' '0' '3' '1']
 ['0' '8' '2' '0' '0' '0' '0'